<a href="https://colab.research.google.com/github/danielmancovidales/Aplicaciones-en-finanzas/blob/main/Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [207]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [208]:
# Lectura de base de datos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [209]:
df= pd.read_csv('/content/drive/MyDrive/analitica3/Financiera/df_final2.csv')
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [210]:
df = df.dropna(subset=['ciudad'])


In [211]:
df.head(2)

,eventos,valor_pagado,ciudad,cancer,epoc,diabetes,hipertension,enf_cardiovascular,genero,edad,cantidad_enfermedades,clasifica_diagn
0,1,715463.04,1.0,0,0,0,0,0,1,27,0,2
1,1,691983.39,1.0,0,1,0,0,0,0,42,1,2


In [212]:
columnas = ['edad','epoc','diabetes','clasifica_diagn','ciudad','hipertension','eventos']

In [213]:
y = df['valor_pagado']
X = df[columnas]

In [214]:
# Si los valores de y son continuos, convertirlos a enteros
if y.dtype != 'int':
    y = y.round().astype(int)



In [215]:
#Comprobar tamaño de variables
print("Tamaño de X :", X.shape)
print("Tamaño de y :", y.shape)

Tamaño de X : (26639, 7)
Tamaño de y : (26639,)


In [216]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [217]:
# Verificar las dimensiones después de la división
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de y_test:", y_test.shape)

Dimensiones de X_train: (21311, 7)
Dimensiones de X_test: (5328, 7)
Dimensiones de y_train: (21311,)
Dimensiones de y_test: (5328,)


In [218]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Ajustar el escalador solo en el conjunto de entrenamiento
X_train = scaler.fit_transform(X_train)

# Transformar el conjunto de prueba
X_test = scaler.transform(X_test)

In [219]:
# Verificar las dimensiones después del preprocesamiento
print("Dimensiones de X_train después del escalado:", X_train.shape)
print("Dimensiones de X_test después del escalado:", X_test.shape)

Dimensiones de X_train después del escalado: (21311, 7)
Dimensiones de X_test después del escalado: (5328, 7)


##Regresión Lineal

In [220]:
# Ajustar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir y evaluar el modelo
y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print('R2:',r2_score(y_test, y_pred))

MSE: 12532974806367.139
R2: 0.014364843042821818


##Regresión Ridge y Lasso

In [221]:
# Ridge Regression
ridge = Ridge(alpha=1.0)
ridge_scores = cross_val_score(ridge, X, y, cv=5, scoring='neg_mean_squared_error')
print("Ridge MSE:", -ridge_scores.mean())

# Lasso Regression
lasso = Lasso(alpha=0.1)
lasso_scores = cross_val_score(lasso, X, y, cv=5, scoring='neg_mean_squared_error')
print("Lasso MSE:", -lasso_scores.mean())
print('R2:',r2_score(y_test, y_pred))

Ridge MSE: 9317832220288.969
Lasso MSE: 9317856419263.73
R2: 0.014364843042821818


##Random Forest

In [222]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión con Random Forest
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

# Entrenar el modelo
rf_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_regressor.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)

MSE: 11546887274669.559
RMSE: 3398071.1108906413
MAE: 1705614.4325353429
R-squared: 0.09191407250305306


##XGBoost

In [223]:
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))

MSE: 11264536117128.057


In [224]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión con Decision Tree Regressor
dt_regressor = DecisionTreeRegressor()

# Entrenar el modelo
dt_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = dt_regressor.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)


MSE: 12117393851675.605
RMSE: 3481004.718709184
MAE: 1757534.131851207
R-squared: 0.04704752260088729


In [225]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos de entrada
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Crear el modelo de regresión con SGDRegressor
sgd_regressor = SGDRegressor()

# Entrenar el modelo
sgd_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = sgd_regressor.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)


MSE: 12580419377994.836
RMSE: 3546888.689823073
MAE: 1880967.47420107
R-squared: 0.010633642866844983
